## 1. Get MassIVE

In [ ]:
from filter_datasets import *
import json
import ppx #A Python interface to proteomics data repositories
from pyteomics import mztab
from ete3 import Tree, NodeStyle, TreeStyle, NodeStyle, faces, AttrFace, CircleFace
from ete3 import NCBITaxa

In [ ]:
databases_massIVE = getAllDts() #get all datesets available on massIVE
MassIVE_datasets_df = pd.DataFrame(databases_massIVE)
MassIVE_datasets_df.species[MassIVE_datasets_df.species.str.contains('3702')] # Arabidopsis positive control. Search made through taxon ID
MassIVE_datasets_df.head()

In [ ]:
MassIVE_datasets_df.columns

In [ ]:
# massive_plantid_proteomics.tsv is the result from filterMassIVE function on MassIVE databases
MassIVE_passed_df = pd.read_csv('massive_plantid_proteomics.tsv', sep='\t')
MassIVE_passed_df.head()

## 2. GET PRIDE

In [ ]:
# pride_plant.json is the result from filterPride function on PRIDE databases
with open('pride_plant.json') as pride_plants_file:
    pride_plant = json.load(pride_plants_file)
print(f"Total of plants retrieved from pride database: {len(pride_plant)}")

### Tiago

In [ ]:
# import json
# with open("pride_projects.json","w+") as pride_projets:
remote_files = []

for projetc_identifiers,accession_number in pride_plant.items():
 if findmztab(projetc_identifiers) != None:
    retrivied_projects = ppx.find_project(projetc_identifiers,local = "/home/tiago/documents/lncRNA/pride_projects")
    remote_files.append(retrivied_projects.remote_files())
print(remote_files)




            
            # json.dump(retrivied_projects,pride_projets)
            #  for projects,mztable_links in retrivied_projects.items():
            #      print(mztable_links)

            #  print(retrivied_projects)


            #  for retrivied_projects.items()
            #  print(retrivied_projects.values())
        


In [ ]:
remote_files

### Ricardo

In [ ]:
projetc_identifiers = [] # holds project number for all plants species data from pride
accession_number = [] # holds taxon id for all plants species data from pride
for key,value in pride_plant.items():
    projetc_identifiers.append(key)
    accession_number.append(value['accession'])
    

In [ ]:
projetc_identifiers[:5]

In [ ]:
accession_number[:5]

In [ ]:
proj = ppx.find_project(projetc_identifiers[0]) #Get files from projects  
remote_files = proj.remote_files()
print(remote_files)

In [ ]:
findmztab(projetc_identifiers[0])==None # select m/z tables from a given project.  

In [ ]:
#Search for project with mztab in pride plants database
for i in projetc_identifiers:
    if findmztab(i)!= None: #Selec only those files with m\z tables
        print(i)
        break #stops at fist result 

In [ ]:
#Explores the first result
#Everything from here must be done to all findings 
proj = ppx.find_project('PRD000084')

In [ ]:
proj.metadata.keys()

In [ ]:
pride_plant['PRD000084']

In [ ]:
remote_files = proj.remote_files()
print(remote_files)

In [ ]:
findmztab(i)

In [ ]:
downloaded = proj.download("generated/PRIDE_Exp_Complete_Ac_15505.pride.mztab.gz")
print(downloaded)

In [ ]:
!gzip -df /home/tiago/.ppx/PRD000084/generated/PRIDE_Exp_Complete_Ac_15505.pride.mztab.gz 

In [ ]:
!ls /home/tiago/.ppx/PRD000084/generated/

In [ ]:
tables = mztab.MzTab("/home/tiago/.ppx/PRD000084/generated/PRIDE_Exp_Complete_Ac_15505.pride.mztab")
psms = tables.spectrum_match_table
psms.head()

In [ ]:
tables.keys()

In [ ]:
prt = pd.DataFrame(tables['PRT'])

In [ ]:
prt.head()

In [ ]:
prt['description'].str.contains('unknown').sum() #Number of unknown proteins confirmed 

In [ ]:
prt["description"].str.contains("putative").sum() #Number of putative proteins 

In [ ]:
prt["description"].str.contains("hypothetical").sum() #Nuber of hypothetical proteins

In [ ]:
prt['description'].str.contains('lnc').sum() #Number of long non coding RNA?

In [ ]:
prt['description'].str.contains('coding', case=False).sum() #Number of coding RNA?

In [ ]:
prt['description'].str.contains('non', case=False).sum() #Number of coding RNA

In [ ]:
idx = prt['description'].str.contains('unknown') #see the proteins found
idx

In [ ]:
idx[0] = False
prt[idx]

In [ ]:
accession = list(set(accession))

print(len(accession))
accession[:10]

In [ ]:
ncbi = NCBITaxa()


tree = ncbi.get_topology(accession[:3])

print (tree.get_ascii(attributes=["sci_name", "rank"]))

# Not working, make plot of species available, can use other tools
# https://phylot.biobyte.de
ts = TreeStyle()
ts.show_leaf_name = True
ts.mode = "c"
ts.arc_start = -180 # 0 degrees = 3 o'clock
ts.arc_span = 180

for n in tree.traverse():
    if n.is_leaf():
        n.name = n.sci_name

tree.render("pride_plant_tree.pdf", w=500, tree_style=ts)